<a href="https://colab.research.google.com/github/AlineVileforte/Redes_Neurais/blob/master/Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

bibliotecas

In [18]:
!pip install numpy matplotlib tensorflow


Importações

In [19]:
import os
import gzip
import urllib.request
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

URLs dos arquivos MNIST originais

In [20]:
base_url = "https://ossci-datasets.s3.amazonaws.com/mnist/"
files = {
    "train_images": "train-images-idx3-ubyte.gz",
    "train_labels": "train-labels-idx1-ubyte.gz",
    "test_images": "t10k-images-idx3-ubyte.gz",
    "test_labels": "t10k-labels-idx1-ubyte.gz"
}


Baixar e extrair arquivos .gz

In [21]:
os.makedirs("mnist_data", exist_ok=True)
for name, filename in files.items():
    path = os.path.join("mnist_data", filename)
    if not os.path.exists(path):
        print(f"Baixando {filename}...")
        urllib.request.urlretrieve(base_url + filename, path)

Funções de leitura do formato binário do MNIST

In [22]:
def read_labels(path):
    with gzip.open(path, 'rb') as f:
        f.read(8)  # Ignora o cabeçalho
        return np.frombuffer(f.read(), dtype=np.uint8)

def read_images(path):
    with gzip.open(path, 'rb') as f:
        f.read(16)  # Ignora o cabeçalho
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data.reshape(-1, 28, 28, 1).astype('float32') / 255.0


carregar dados

In [23]:
X_train = read_images("mnist_data/train-images-idx3-ubyte.gz")
y_train = read_labels("mnist_data/train-labels-idx1-ubyte.gz")
X_test = read_images("mnist_data/t10k-images-idx3-ubyte.gz")
y_test = read_labels("mnist_data/t10k-labels-idx1-ubyte.gz")

 One-hot encoding das classes

In [24]:
num_classes = 10
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)

Construção da CNN com TensorFlow/Keras


In [25]:
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D(pool_size=2),
    layers.Conv2D(64, kernel_size=3, activation='relu'),
    layers.MaxPooling2D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

definir rede neural convolucional

compilar modelo

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


treinando

In [27]:
model.fit(X_train, y_train_cat, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 44s 49ms/step - accuracy: 0.8776 - loss: 0.4070 - val_accuracy: 0.9837 - val_loss: 0.0561
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 80s 48ms/step - accuracy: 0.9843 - loss: 0.0525 - val_accuracy: 0.9885 - val_loss: 0.0412
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - accuracy: 0.9902 - loss: 0.0339 - val_accuracy: 0.9878 - val_loss: 0.0419
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - accuracy: 0.9924 - loss: 0.0240 - val_accuracy: 0.9917 - val_loss: 0.0298
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 42s 48ms/step - accuracy: 0.9942 - loss: 0.0178 - val_accuracy: 0.9912 - val_loss: 0.0327


avaliando

In [28]:
loss, acc = model.evaluate(X_test, y_test_cat)
print(f"\nAcurácia no conjunto de teste: {acc * 100:.2f}%")


313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9883 - loss: 0.0333

Acurácia no conjunto de teste: 99.08%
